In [ ]:
# pymysql을 이용하면 파이썬에서 SQL을 직접 수행할 수 있다

# 다음은 매일 일정한 시간에 네이버 금융 데이터를 스크랩해서 MariaDB에 업데이트하는 DBUpdater 클래스다

# 여기 클래스로 이름붙은 애들 잘 보자, 얘들은 전부 클래스이름별로 각 파일로 빠져나가야할 애들이다
# 지금 작성하는 이 주피터는 그냥 공부용 책으로 생각하면 된다.
# 작성 내용은 책과 동일하게 맞출 것이고, 실제 DB CRUD내용은 커스텀해서 파일로 만들어나가자

# 여기를 실행시키려면 cmd에서 python ~/DBUpdater.py 하면 되고, 테이블 확인은 MariaDB에서 직접 조회하자

# 참고로 아래의 클래스 함수들은 책에서 소개한 순서대로 작성한 것이며
# 순차적으로 읽어나가면 된다

'''
class DBUpdater:
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성"""
    
    def __del__(self):
        """소명자" MariaDB 연결 해제"""
    
    def red_krx_code(self):
        """KRX로 부터 상장법인 목록 파일을 읽어와서 DataFrame으로 변환"""
    
    def update_comp_info(self):
        """종목코드를 company_info 테이블에 업데이트한 후 딕셔너리에 저장"""
    
    def read_naver(self, code, company, pages_to_fetch):
        """네이버 금융에서 주싟 시세를 읽어서 DataFrame으로 반환"""
    
    def replace_into_db(self, df, num, code, company):
        """네이버 금융에서 읽어온 주식 시세를 DB에 REPLACE"""

    def update_daily_price(self, pages_to_fetch):
        """KRX 상장법인의 주식 시세를 네이버로부터 읽어서 DB에 업데이트"""
    
    def execute_daily(self):
        """실행 즉시 및 매일 오후 다섯시에 daily_price 테이블 업데이트"""

if __name__ == '__main__':
    dbu = DBUpdater()
    dbu.execute_daily()
'''
import pymysql
import pandas as pd
import requests
from datetime import datetime

import pandas as pd
from bs4 import BeautifulSoup
import urllib, pymysql, calendar, time, json
from urllib.request import urlopen
from datetime import datetime
from threading import Timer
from pandas import concat

class DBUpdater:
    def __init__(self): # 2. MariaDB와 연결되는 생성자 부분
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성""" # charset utf8로 해주는건 컬럼에 한글 들어갈 때 오류를 피하기 위함
        self.conn = pymysql.connect(host='localhost', port=3306, db='Investar', user='root', passwd='@@@@@@@', charset='utf8')

        with self.conn.cursor() as curs:
            sql = """
            CREATE TABLE IF NOT EXISTS COMPANY_INFO
            (
                CODE VARCHAR(20),
                COMPANY VARCHAR(20),
                LAST_UPDATE DATE,
                PRIMARY KEY (CODE)
            )
            """
            curs.execute(sql)


            sql = """
            CREATE TABLE IF NOT EXISTS DAILY_PRICE
            (
                CODE VARCHAR(20),
                DATE DATE
                OPEN BIGINT(20),
                HIGH BITINT(20),
                LOW BIGINT(20),
                CLOSE BIGINT(20),
                DIFF BIGINT(20),
                VOLUME BIGINT(20),
                PRIMARY KEY (CODE, DATE)
            )
            """
            curs.execute(sql)

        self.conn.commit()

        self.codes = dict()
        self.update_comp_info() # KRX 주식 코드를 읽어와서 COMPANY_INFO 테이블에 업데이트하는 것

    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
        self.conn.close()
    
    def red_krx_code(self): # KRX로 부터 상장법인 목록 파일을 읽어온다
        """KRX로 부터 상장법인 목록 파일을 읽어와서 DataFrame으로 변환"""

        url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage#'

        # read_html로 읽어들이고
        krx = pd.read_html(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text)[0]

        # 종목코드와 회사명 두개의 컬럼만 남긴다
        # [[ , , , ]] 이런식으로 활용하면 원하는 컬럼을 순서대로 재구성 할 수 있다.
        # ( []하면 Series로 출력되고 [[]] 스타일이 DataFrame으로 출력되는 방식이었던 걸로 기억한다.
        krx = krx[['종목코드', '회사명']]

        # 컬럼이름을 변경하는 함수는 다음과 같다
        krx = krx.rename(columns={'종목코드':'CODE', '회사명':'COMPANY'})

        # 종목코드 앞자리 00~ 들을 패딩해주고 총 6자리로 맞춤
        krx.code = krx.code.map('{:06d}'.format)

        return krx
    
    def update_comp_info(self):
        """종목코드를 company_info 테이블에 업데이트한 후 딕셔너리에 저장"""
        
        sql =  "SELECT * FROM COMPANY_INFO"

        df = pd.read_sql(sql, self.conn) # SQL을 일단 읽음

        for idx in range(len(df)):
            self.codes[df['CODE'].values[idx]] = df['COMPANY'].values[idx] # 위에서 읽은 내용으로 codes라는 딕셔너리에 종목과 회사이름 넣음
        
        with self.conn.cursor() as curs:
            sql = "SELECT MAX(LAST_UPDATE) FROM COMPANY_INFO"
            curs.execute(sql)
            rs = curs.fetchone() # DB에서 가장 최근 업데이트 읽자를 가져온다.
            today = datetime.today().strftime('%Y-%m-%d')

            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today: # 위에서 구한 마지막 업데이트일자가 없거나, 오늘보다 과거인 경우만 업데이트 한다.
                krx = self.read_krx_code()

                for idx in range(len(krx)):
                    code = krx.code.values[idx]
                    company = krx.company.values[idx]
                    sql = f"""
                    REPLATE INTO COMPANY_INFO
                    (
                        CODE,
                        COMPANY,
                        LAST_UPDATE
                    )
                    VALUES
                    (
                        {code},
                        {company},
                        {today}
                    )
                    """
                    curs.execute(sql) # REPLACE INTO를 활용해서 종목코드, 회사명, 오늘날짜 행들을 DB에 저장한다

                    self.codes[code] = company # codes 딕셔너리에 키-값 쌍으로 종목코드와 회사명을 추가한다

                    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')

                    print(f"[{tmnow}] {idx:04d} REPLACE INTO COMPANY_INFO COMPANY "\
                    f"VALUES ({code}, {company}, {today}")
                
                self.conn.commit()
                print(' ')
        
        if __name__ == '__main__':
            dbu = DBUpdater()   # 1. DBUpdate.py가 단독으로 실행되면 DBUpdater 객체를 생성한다
            dbu.update_comp_info() # 3. COMPANY_INFO 테이블에 오늘 업데이트된 내용이 있는지 확인 하고
                                    #    없으면 4를 호출하면 COMPANY_INFO 테이블을 업데이트하고 codes 딕셔너리에도 저장한다.
            dbu.execute_daily() 

        
        # 위쪽이 KRX 종목코드를 DB에 업데이트하는 부분이었다면
        # 아래쪽은 네이버에서 스크랩하는 내용
        # 특히 핵심은 pgRR 클래스의 <td> 태그가 없으면
        # AttributeError 발생하면서 프로그램이 종료되니, find() 함수 결과가 None일 땐 다음 종목을 처리토록 한다
        
        def read_naver(self, code, company, pages_to_fetch):
            """ 네이버에서 시세를 읽고 DataFrame으로 반환"""
                        
            try:
                url = f'https://finance.naver.com/item/sise_day.naver?code={code}'

                html = BeautifulSoup(requests.get(url, headers={"User-agent: Mozilla/5.0"}).txt, "lxml")
                
                pgrr = html.find('td', class_='pgRR')

                if pgrr is None:
                    return None

                s = str(pgrr.a['href']).split('=')
                last_page = s[-1] # 네이버금융에서 시세의 마지막 페이지를 구한다

                df = pd.DataFrame()

                pages = min(int(last_page), pages_to_fetch) # 설정파일에서 설정된 페이지 수(pages_to_fetch)와 last_page 중 작은 것을 택함

                for page in range(1, pages + 1):
                    page_url = '{}&page={}'.format(url, page)
                    df = pd.concat([df, pd.read_html(requests.get(page_url, headers={'User-agent': 'Mozilla/5.0'}).text)[0]])

                    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')

                    print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.format(tmnow, company, code, page, pages), end="\r")

                # 네이버 금융에서 받은 자료의 컬럼이름들을 영문으로 변환
                df = df.rename(columns={'날짜':'date', '시가':'open', '고가':'high', '저가':'low', '종가':'close', '거래량':'volume'})
                # 날짜 표기 형식도 변환
                df['date'] = df['date'].replace('.', '-')
                df = df.dropna()

                # MariaDB의 BIGINT형 컬럼들에 들어갈 데이터들을 int형으로 변환 해 줌
                df[['close','diff','open','high','low','volume']] = df[['close','diff','open','high','low','volume']].astype(int)

                # 원하는 순서로 컬럼을 재조합하여 DataFrame 생성
                df = df[['date','open','high','low','close','diff','volume']]
            
            except Exception as e: # HTML을 받아오는 때에 에러가 발생할 수 있어서 예외처리로 프로그램 종료를 방지한다
                print("Exception Occured : ", str(e))
                return None
            
            return df


    def replace_into_db(self, df, num, code, company):
        """네이버에서 읽어온 주식 시세를 DB에 REPLACE INTO한다"""
        
        with self.conn.cursor() as curs:
            for r in df.itertuples(): # 넘겨받은 DataFrame을 순회처리한다
                sql = f""""
                      REPLACE INTO DAILY_PRICE 
                      VALUES 
                      (
                      {code},
                      {r.date},
                      {r.open},
                      {r.high},
                      {r.low},
                      {r.close},
                      {r.diff},
                      {r.volume}
                      )
                      """
                curs.execute(sql) # 위의 쿼리를 돌려 줌
            
            self.conn.commit() # commit해서 DB에 반영

            print('[{}] %{:04d} {} ({}) : {} rows > REPLACE INTO DAILY_PRICE [OK]'\
                .format(datetime.now().strftime('%Y-%m-%d %H:%M'), num+1, company, code, len(df)))
    
    def update_daily_price(self, pages_to_fetch):
        """KRX 상장법인의 주식 시세를 네이버로부터 읽어서 DB에 업데이트"""
        for idx, code in enumerate(self.codes): # self.codes 딕셔너리에 저장된 모든 종목코드에 대해 순회처리 한다
            df = self.read_naver(code, self.codes[code], pages_to_fetch) # read_naver 메서드를 이용하여 종목코드에 대한 일별시세 DataFrame을 구한다
            if df is None:
                continue
            self.replace_into_db(df, idx, code, self.codes[code]) # 위에서 구한 DataFrame을 replace_into_db로 DB에 저장한다

    # config.json 파일을 이용해 DBUpdater의 과거 실행여부를 체크한다
    # 이 파일이 없으면 DBUpdater가 처음 실행되는 경우이므로
    # 네이버 시세 데이터를 종목별로 100페이지씩 가져온다.
    # 최초 업데이트 이후 부터는 1페이지씩 가져오도록 
    # config.json파일이 자동으로 변경된다
    # 만약 업데이트할 페이지 수를 변경하려면 config.json 파일의 pages_to_fetch값을 수정하면 된다

    # 네이버 시세 데이터는 한페이지에 10일치 종가 데이터를 제공하므로
    # 100페이지는 1000일의 데이터다
    # 1년에 250일 정도 개장하므로, 100페이지는 4년치 데이터로 본다

    # execute_daily() 메서드는 DBUpdater.py 모듈의 시작 포인트다
    # 타이머가 매일 오후 5시에 호출된다.

    def execute_daily(self):
        """실행 즉시 및 매일 오후 5시에 daily_price 테이블 업데이트"""
        self.update_comp.info() # 상장법인목록 DB를 업데이트 한다

        try:
            with open('config.json', 'r') as in_file: # DBUpdater.py가 있는 경로에서 config.json파일을 읽는다
                config = json.load(in_file)
                pages_to_fetch = config['pages_to_fetch']
        except FileNotFoundError: # config.json 파일이 없으면 최초 pages_to_fetch를 100으로 설정한다
            with open('config.json', 'w') as out_file:
                pages_to_fetch = 100
                config = {'pages_to_fetch':1}
                json.dump(config, out_file)
        
        self.update_daily_price(pages_to_fetch) # 설정된 pages_to_fetch를 이용해 메소드 실행

        tmnow = datetime.now()
        lastday = calendar.monthrange(tmnow.year, tmnow.month)[1] # 이번달 마지막날을 구해서 다음날 오후 5시를 계산하는데 사용

        if tmnow.month == 12 and tmnow.day == lastday:
            tmnext = tmnow.replace(year=tmnow.year+1, month=1, day=1, hour=17, minute=0, second=0)
        elif tmnow.day == lastday:
            tmnext = tmnow.replace(month=tmnow.month+1, day=1, hour=17, minute=0, second=0)
        else:
            tmnext = tmnow.replace(day=tmnow.day+1, hour=17, minute=0, second=0)
        
        tmdiff = tmnext - tmnow
        secs = tmdiff.seconds

        t = Timer(secs, self.execute_daily) # 다음 날 오후 5시에 execute_daily()를 실행하는 Timer 인스턴스를 생성
        print("Waiting for next update ({}) ...".format(tmnext.strftime('%Y-%m-%d %H:%M')))

        t.start()

    # 매일 오후 5시 정각에 DB를 업데이트하려면 MariaDB 설정 파일에서 wait_timeout값을 변경해야한다
    # 이걸하지 않으면 DBUpdater의 생성자에서 connect()함수로 MariaDB 연결한 뒤 8시간 이상 쓰지 않으면
    # 세션이 종료되기 때문이다

    # MariaDB 설정파일 ( my.ini 에서 [mtsqld] 아래에 wait_timeout=288000 을 추가해서 장시간 세션이 끊기지 않도록 하자


    # 서버가 재시작 하더라도 위의 프로그램이 재실행되게 하려면 Run 레지스트리에 등록해야한다
    # DBUpdater.bat이란 파일을 만들고, 메모장으로 열어서
    # python C:\~\DBUpdater.py 라고 만들자
    # 그뒤 Win+R -> regedit로 가서
    # HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Windows\CurrentVersion\Run 키를 우클릭 한 뒤
    # 새로만들기-> 문자열 값 메뉴를 클릭해서 DBUPDATER라는 값으로 위에서 만든 bat파일을 등록해주면
    # 이 배치가 pc 재기동 때마다 항상 재시작 된다
